<a href="https://colab.research.google.com/github/shiehn/dawnet-remotes/blob/main/DAWNet_Remote_Audio_2_MIDI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Check if ffmpeg is installed, if not then install it
ffmpeg_installed = !command -v ffmpeg
if not ffmpeg_installed:
    !apt-get install ffmpeg

#Check if BASIC_PITCH is installed, if not then install it
try:
  from basic_pitch.inference import predict_and_save
  from basic_pitch import ICASSP_2022_MODEL_PATH
except ImportError:
  !pip install basic-pitch

#LOAD THE BASIC_PITCH MODEL
basic_pitch_model = tf.saved_model.load(str(ICASSP_2022_MODEL_PATH))

#Check if dawnet-client is installed, if not then install it
try:
    import dawnet_client
except ImportError:
    !pip install dawnet-client
    import dawnet_client

try:
    import dawnet_client
except ImportError:
  !pip install dawnet-client --upgrade


import dawnet_client.core as dawnet
from dawnet_client import DAWNetFilePath

placeholder_txt = "Enter the token generated by the DAWNet plugin"
DAWNET_TOKEN = "Enter the token generated by the DAWNet plugin" #@param {type:"string"}
dawnet_token = DAWNET_TOKEN

if dawnet_token is None or dawnet_token == "" or dawnet_token == placeholder_txt:
  print("ERROR: The token provided is not valid.")
  exit()

async def dawnet_func(input_file: DAWNetFilePath):
    try:
        # Create a temporary directory
        temp_dir = tempfile.mkdtemp()

        #Extract MIDI
        predict_and_save(
            audio_path_list=input_file,
            output_directory=temp_dir,
            save_midi=True,
            sonify_midi=False,
            save_model_outputs=False,
            save_notes=False,
            model_path=basic_pitch_model
            midi_tempo=round(dawnet.get_daw_bpm()))


        # Get the list of file and directory names in the given directory
        contents = os.listdir(temp_dir)
        contents_string = ', '.join(contents)

        # Print the contents of the directory
        print(contents_string)

        # after executing your custom code you send data back to the plugin like so ..
        await dawnet.output().add_file(input_file)
        await dawnet.output().add_message(contents_string)
        await dawnet.output().send()

        return True
    except Exception as e:
        await dawnet.output().add_error(f"Error in arbitrary_method: {e}")

        return False


dawnet.set_token(token=dawnet_token)
dawnet.set_name("DAWNet Template")
dawnet.set_description("This is a template for creating a DAWNet Remote")
dawnet.register_method(dawnet_func)

dawnet.connect_to_server()